<a href="https://colab.research.google.com/github/Zezo-Elkafoury/Food-Classifier-API-DL/blob/main/Food_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Loading Data

In [ ]:
data=load_dataset('food101',split='train[:5000]')

### Inspecting the data

In [ ]:
data

## Splitting Data

In [ ]:
data=data.train_test_split(test_size=0.2)

In [ ]:
data

In [ ]:
data['train'][0]['image']

## Mapping Labels

In [ ]:
labels=data['train'].features['label'].names
label2id,id2label={},{}
for i, label in enumerate(labels):
    label2id[label]=i
    id2label[i]=label

In [ ]:
id2label

In [ ]:
label2id

# Preprocessing Images

In [ ]:
! pip install --upgrade transformers

In [ ]:
from transformers import AutoImageProcessor

In [ ]:
processor=AutoImageProcessor.from_pretrained('google/vit-base-patch16-224') #importing the processing needed for the model

Getting image dimensions for the pretrained model

In [ ]:
size=(processor.size['height'],processor.size['width'])
size

In [ ]:
!pip install torchvision

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
transform=Compose([
    RandomResizedCrop(size),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [ ]:
def proccess_images(samples):
  samples['pixel_values']=[transform(img.convert('RGB')) for img in samples['image'] ]
  del samples['image']
  return samples

In [ ]:
data_final=data.with_transform(proccess_images) # applying transformation on the data

# Model Training

In [ ]:
from transformers import AutoModelForImageClassification, Trainer, TrainingArguments
model= AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224',
                                                            num_labels=len(labels),
                                                            id2label=id2label,
                                                            label2id=label2id,
                                                            ignore_mismatched_sizes=True)

In [ ]:
from transformers import DefaultDataCollator
data_collator=DefaultDataCollator()

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import numpy as np
metric=evaluate.load('accuracy')
def compute_metrices(predictions_and_labels):
  predictions,labels=predictions_and_labels
  predictions=np.argmax(predictions,axis=1)
  return metric.compute(predictions=predictions,references=labels)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args=TrainingArguments(
    output_dir='./food_model',
    run_name='food_classification_experiment',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=10,
    metric_for_best_model='accuracy',
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=True, # To publish the model on huggingface
    remove_unused_columns=False,
    load_best_model_at_end=True
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=data_final['train'],
    eval_dataset=data_final['test'],
    processing_class=processor,
    compute_metrics=compute_metrices,
)

In [ ]:
trainer.train()

In [ ]:
import matplotlib.pyplot as plt

# Initialize lists
train_loss = []
train_steps = []

eval_loss = []
eval_accuracy = []
eval_steps = []

# Extract values
for log in trainer.state.log_history:
    if 'loss' in log and 'step' in log:
        train_loss.append(log['loss'])
        train_steps.append(log['step'])
    if 'eval_loss' in log:
        eval_loss.append(log['eval_loss'])
        eval_accuracy.append(log['eval_accuracy'])
        eval_steps.append(log['step'])


plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(train_steps, train_loss, label='Training Loss', color='blue')
plt.plot(eval_steps, eval_loss, label='Validation Loss', color='red')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Accuracy plot
plt.subplot(1, 2, 2)
plt.plot(eval_steps, eval_accuracy, label='Validation Accuracy', color='green')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Inference System and API

In [ ]:
!pip install FastAPI
!pip install uvicorn
!pip install python-multipart
!pip install aiofiles

In [ ]:
from fastapi import FastAPI, File, UploadFile
from typing import List
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
import io

app = FastAPI()

# Load the pre-trained processor and model
processor = AutoImageProcessor.from_pretrained("ZiadElkafoury/food_model")
model = AutoModelForImageClassification.from_pretrained("ZiadElkafoury/food_model")


@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    """
    Predicts the food item in an uploaded image.

    Args:
        file (UploadFile): The image file to be processed.

    Returns:
        dict: A dictionary containing the predicted label.
    """
    try:
        # Read the image file
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("RGB")

        # Preprocess the image
        inputs = processor(images=image, return_tensors="pt")

        # Make the prediction
        with torch.no_grad():
            outputs = model(**inputs)

        # Get the predicted label
        predicted_label_id = outputs.logits.argmax(-1).item()
        predicted_label = model.config.id2label[predicted_label_id]

        # Return the prediction
        return {"predicted_label": predicted_label}

    except Exception as e:
        return {"error": str(e)}